### Desafío Data Science - Juliana Plácido

#### Notebook III
Baseline Model

In [35]:
#Importamos librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
#from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit


import gc
import xlsxwriter
import time
import warnings
warnings.filterwarnings('ignore')

OSError: dlopen(/Users/jplacido/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/jplacido/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

In [3]:
#importamos el df
df = pd.read_csv('df_modelado.csv')

In [21]:
X = df.drop(['vlo-i','fecha-i', 'atraso_15', 'flag_adelantado'], axis=1)
y = df['atraso_15']

In [22]:
from sklearn.model_selection import train_test_split
trainSize = 0.7

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,
                                  train_size=trainSize)

In [23]:
print('Shape X:', X_train.shape)
print('Shape y:', y_train.shape)

Shape X: (47716, 15)
Shape y: (47716,)


In [24]:
print('Shape X:', X_test.shape)
print('Shape y:', y_test.shape)

Shape X: (20450, 15)
Shape y: (20450,)


In [33]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

#Predict
y_train_pred = model.predict_proba(X_train)[:,1]
y_test_pred = model.predict_proba(X_test)[:,1]

train_roc = round(roc_auc_score(y_train, y_train_pred) ,4) 
test_roc = round(roc_auc_score(y_test, y_test_pred) ,4)

print("Logistic Reg Baseline Train: " + str(train_roc))
print("Logistic Reg Baseline Test: " + str(test_roc))

Logistic Reg Baseline Train: 0.6205
Logistic Reg Baseline Test: 0.6184


In [28]:
### Probamos un arbol de clasificación como benchmark
#Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
#Predict
y_train_pred = tree.predict_proba(X_train)[:,1]
y_test_pred = tree.predict_proba(X_test)[:,1]

train_roc = round(roc_auc_score(y_train, y_train_pred) ,4) 
test_roc = round(roc_auc_score(y_test, y_test_pred) ,4)

print("Tree Classifier Baseline Train: " + str(train_roc))
print("Tree Classifier Baseline Test: " + str(test_roc))

Tree Classifier Baseline Train: 0.9891
Tree Classifier Baseline Test: 0.5942


In [30]:
#Random Forest Classifier como benchmark 2
from sklearn.ensemble import RandomForestClassifier
trees = RandomForestClassifier(random_state=42)
trees.fit(X_train, y_train)
#Predict
y_train_pred = trees.predict_proba(X_train)[:,1]
y_test_pred = trees.predict_proba(X_test)[:,1]

train_roc = round(roc_auc_score(y_train, y_train_pred) ,4) 
test_roc = round(roc_auc_score(y_test, y_test_pred) ,4)

print("Tree Classifier Baseline Train: " + str(train_roc))
print("Tree Classifier Baseline Test: " + str(test_roc))

Tree Classifier Baseline Train: 0.9881
Tree Classifier Baseline Test: 0.6746


In [31]:
%%time
model_xgboost = XGBClassifier(random_state=42)
model_xgboost.fit(X_train, y_train)

preds_xgb_train = model_xgboost.predict_proba(X_train)[:,1]
preds_xgb_test = model_xgboost.predict_proba(X_test)[:,1]

train_roc = round(roc_auc_score(y_train, preds_xgb_train) ,4) 
test_roc = round(roc_auc_score(y_test, preds_xgb_test) ,4)

print("ROC XGBoost Baseline Train: " + str(train_roc))
print("ROC XGBoost Baseline Test: " + str(test_roc))

ROC XGBoost Baseline Train: 0.8273
ROC XGBoost Baseline Test: 0.7321
CPU times: user 11.2 s, sys: 934 ms, total: 12.1 s
Wall time: 1.59 s


In [32]:
feat_imp_xgb = pd.DataFrame()
feat_imp_xgb['Variable'] = X_train.columns
feat_imp_xgb['importance'] = model_xgboost.feature_importances_
feat_imp_xgb = feat_imp_xgb.sort_values(by='importance', ascending=False)
feat_imp_xgb.head(12)

,Variable,importance
4,tipo_vuelo,0.256244
5,periodo_dia,0.073749
14,cod_aerolinea,0.070424
1,mes,0.068983
8,Lunes,0.063576
13,cod_destino,0.060215
12,Viernes,0.054289
9,Martes,0.050937
0,dia,0.047280
7,Jueves,0.047220
